L'explication du script se fera de façon chronologique pour faciliter la compréhension, certaines fonctions seront ainsi expliquées après le script principal

## Creation bdd

Se trouve dans le fichier creation_bdd.py.
C'est un script simple permettant la création en externe de la base de données avec les colonnes et le type de données (à optimiser), cela permet de : 
- Ne pas créer la base de données en se basant sur le premier xml traité, ce qui pose problème lorsqu'un futur acte budgétaire contiendra plus d'informations (colonnes)
- Pré-structurer les dates en date, certaines version du schema ne conservent pas les dates au même endroit ou de la même façon créant des conflits
- Faciliter l'expérimentation du code

In [ ]:
import sqlite3

BDD = 'bdd_actes_budgetaires_gz.db'

def creation_bdd() : 
 ''' Crée une base de données contenant les colonnes que nous allons chercher dans le script etl'''
 conn = sqlite3.connect(BDD)
 cursor = conn.cursor()
 cursor.execute('''
  CREATE TABLE IF NOT EXISTS acte_budgetaire_gz ( 
    VersionSchema INT,
    DteDec DATE,
    LibelleColl TEXT,
    IdColl INT,
    Nature INT,
    LibCpte TEXT,
    Fonction INT,
    Operation INT,
    ContNat INT,
    ArtSpe BOOLEAN,
    ContFon INT,
    ContOp INT,
    CodRD TEXT,
    MtBudgPrec INT,
    MtRARPrec INT,
    MtPropNouv REAL,
    MtPrev REAL,
    CredOuv REAL,
    MtReal REAL,
    MtRAR3112 INT,
    OpBudg INT,
    TypOpBudg INT,
    OpeCpteTiers INT
                ) 
                ''')
 conn.commit()
 conn.close()


if __name__ == "__main__":
    creation_bdd()

## Etl_Fichier_GZ

Décomposé en de nombreuses petites fonctions, le processus se fait de la façon suivante : 
- le script va chercher les fichiers xml.gz dans le dossier "DOSSIER_SOURCE"
- Extraire les données que nous cherchons dans chaque fichier sous forme de df
- Agglomérer les différents DataFrame (un par fichier)
- Transformer les données afin de se débarasser des @V etc.
- Déplacer les fichiers traités dans le dossier "DOSSIER_SORTIE"
- Insérer le DataFrame aggloméré dans la base de données "bdd_actes_budgetaires_gz.db", dans le dossier "DOSSIER_PARENT"
- Créer une copie de la base de données sous forme de csv


In [ ]:
DOSSIER_PARENT = r" "
DOSSIER_SOURCE = r"./traitement en cours/"
DOSSIER_SORTIE = r"./traite/"
BDD = 'bdd_actes_budgetaires_gz.db'
NOM_CSV = 'donnees_budgetaires.csv'

TIMING_DECORATOR

Optionnel, permet de calculer la vitesse des fonctions pour chercher des optimisations.

In [ ]:
def timing_decorator(func):
 def wrapper(*args, **kwargs):
  start_time = time.time()
  result = func(*args, **kwargs)
  end_time = time.time()
  execution_time = end_time - start_time
  print(f"{func.__name__} a pris {execution_time:.4f} secondes pour s'exécuter.")
  return result
 return wrapper

OUVERTURE_GZIP

Cherche le fichier .xml.gz, l'ouvre, le décode pour permettre sa lisibilité et transforme son contenu en dictionnaire

In [ ]:
def ouverture_gzip(chemin) : 
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

PARSING 

Ces fonctions vont individuellement chercher dans le dictionnaire nouvellement créer pour y chercher les informations voulues dans les différents sous dictionnaires : 
Les lignes budgets, l'en tête du document, la version du schema ainsi que la date d'envoi du fichier xml. Ensuite, chaque fonction va créer un mini DataFrame dont les données ne sont pas encore nettoyées (présence de @V etc.)

In [ ]:

def parse_budget(data_dict: dict) -> pd.DataFrame : 
 "Sépare les sous clefs lignes budget, sans nettoyage"
 ligne_budget = data_dict['DocumentBudgetaire']['Budget']['LigneBudget']
 df_ligne_budget = pd.DataFrame(ligne_budget)
 return df_ligne_budget

def parse_metadonnees(data_dict : dict) -> pd.DataFrame : 
 "Sépare les sous clefs de métadonnées, sans nettoyage"
 metadonnees = data_dict['DocumentBudgetaire']['EnTeteDocBudgetaire']
 df_metadonnees = pd.DataFrame(metadonnees)
 return df_metadonnees

def parse_schema(data_dict : dict) -> pd.DataFrame : 
 "Sépare la version schema sans nettoyage"
 version_schema = data_dict['DocumentBudgetaire']['VersionSchema']['@V']
 df_schema = pd.DataFrame({"VersionSchema": [version_schema]})
 return df_schema

def parse_date(data_dict : dict) -> pd.DataFrame : 
 date = data_dict['DocumentBudgetaire']['Budget']['BlocBudget']
 df_date = pd.DataFrame(date)
 return df_date

ASSEMBLAGE

Le but de cette fonction est d'assemblée les différents Dataframes issus des fonctions de parsing, de taille et de contenu différents. 
1. Premièrement, elle crée un DataFrame vide contenant les colonnes voulues (les mêmes que celle de la base de données)
2. Elle "corrige" la taille des différents DataFrame : df_schema, df_date et df_metadonnees ne font qu'une ligne, la fonction 
   va multiplier la taille du dataframe par celle du df_principal (contenant les lignes budget) pour rendre la fusion possible
3. les boucles for ont pour but que le df_final (contenant tout) copie les données de chaque dataframe partageant les colonnes. Cela permet de ne pas récupérer des données dont on ne souhaite pas, par exemple certaines métadonnées qui ont été extraites sans que ce soit intéressant pour nous
4. Optionnel, les colonnes entièrement vides vont être supprimées (pratique pour les tests) 

In [ ]:
def assemblage(df_principal: pd.DataFrame, 
                         df_meta: pd.DataFrame, 
                         df_schem: pd.DataFrame,
                         df_date : pd.DataFrame) -> pd.DataFrame:
 """ Assemble les dataFrame contenant les metadonnees,
 la version schema et les lignes budgetaires """
 colonnes_a_conserver = ["VersionSchema", "DteDec", "LibelleColl",
                         "IdColl","Nature","LibCpte",
                         "Fonction","Operation",
                         "ContNat","ArtSpe",
                         "ContFon", "ContOp",
                         "CodRD","MtBudgPrec",
                         "MtRARPrec","MtPropNouv",
                         "MtPrev","CredOuv",
                         "MtReal","MtRAR3112",
                         "OpBudg","TypOpBudg",
                         "OpeCpteTiers"
                        ]
 
 df_final = pd.DataFrame(columns=colonnes_a_conserver)
 df_schem = pd.concat([df_schem] * len(df_principal), ignore_index=True)
 df_meta = pd.concat([df_meta] * len(df_principal), ignore_index=True)
 df_date = pd.concat([df_date] * len(df_principal), ignore_index=True)

 for col in df_schem.columns:
  if col in colonnes_a_conserver:
    df_final[col] = df_schem[col]

 for col in df_meta.columns:
  if col in colonnes_a_conserver:
    df_final[col] = df_meta[col]

 for col in df_principal.columns:
  if col in colonnes_a_conserver:
    df_final[col] = df_principal[col]

 for col in df_date.columns:
  if col in colonnes_a_conserver:
    df_final[col] = df_date[col]

 df_final = df_final.dropna(axis=1, how='all')
 return df_final

DEPLACEMENT FICHIER

Cette fonction va simplement déplacer, en fin de traitement individuel, le fichier traité du DOSSIER_SOURCE au DOSSIER_SORTIE

In [ ]:
def deplacement_fichier(fichier_a_deplacer, dossier_destination):
 """ Déplace le fichier du dossier source au dossier fini"""
 chemin_source = pathlib.Path(fichier_a_deplacer)
 chemin_destination = pathlib.Path(dossier_destination) / chemin_source.name
 chemin_source.rename(chemin_destination)

FONCTION MAIN partie 1

- La fonction traite, dans la partie 1 les fichiers du DOSSIER_SOURCE un par un. 
- La première boucle for consiste à vérifier que les fichiers dans le DOSSIER_SOURCE ne sont pas déjà dans le DOSSIER_SORTIE (signifiant qu'ils ont déjà été traités),    évitant des doublons dans la base de données. 
- Pour chaque fichier, la fonction main va ouvrir le fichier, le transformer en dict (via ouverture_gzip), extraire et transformer en DataFrame les parties voulues puis assembler ces parties, enfin le fichier sera déplacé dans le DOSSIER_SORTIE, signifiant que ses données ont été extraites. 

In [ ]:
def main():
 """ Traitement global, extrait et transforme les fichiers XML dans DOSSIER_SOURCE
    pour les insérer dans une bdd et en faire un csv"""
 liste_des_fichier_source = glob.glob(os.path.join(DOSSIER_SOURCE, "*.gz"))
 liste_des_fichier_traite = glob.glob(os.path.join(DOSSIER_SORTIE, "*.gz"))
 liste_des_df = []
 for fichier in liste_des_fichier_source:
  logging.info(f'Debut du travail sur {fichier}')
    # Sécurité permettant de ne pas injecter des doublons
  if fichier in liste_des_fichier_traite : 
   logging.error(f'Le fichier {fichier} a déjà été traité')
   return None 
  else : 
   data_dict = ouverture_gzip(fichier)
   if data_dict is not None:
    df_budget = parse_budget(data_dict)
    df_metadonnees = parse_metadonnees(data_dict)
    df_schema = parse_schema(data_dict)
    df_date = parse_date(data_dict)
    if df_budget is not None \
            and df_metadonnees is not None \
            and df_schema is not None \
            and df_date is not None:
     df_final = assemblage(
                df_budget, df_metadonnees, df_schema, df_date)
     liste_des_df.append(df_final)
     logging.info(f'Fin du travail sur {fichier}')
     deplacement_fichier(fichier, DOSSIER_SORTIE)
  #Séparation entre la partie 1 et 2 de la fonction main
 df_session = pd.concat(liste_des_df, ignore_index = True)
 nettoyage_lambda(df_session)
 insertion_bdd(df_session)
 creation_csv()

LISTE_DES_DF & DF_SESSION

A chaque boucle de la partie un, un DataFrame est ajouté à la liste des dataframe, puis ils sont tous agglomérés pour pouvoir être traités. 

NETTOYAGE LAMBDA

En théorie, à cette étape, les fichiers du DOSSIER_SOURCE sont tous traités, les données voulues ne forment qu'un seul DataFrame.
Cette fonction a pour but, d'appliquer colonnes par colonnes une fonction lambda de nettoyage, retirant les '@V: ' des données.  

In [ ]:
def nettoyage_lambda(df : pd.DataFrame) -> pd.DataFrame : 
 "Nettoie les données pour se débarasser des @V"
 nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
 for col in df.columns : 
  df[col] = df[col].apply(nettoyage)
 return df 